refer to https://www.kaggle.com/antonellomartiello/tps09-autogluon

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
import gc
gc.collect()

In [ ]:
pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset(train).drop('Id', axis=1)
train_data.head()


In [ ]:
test_data = TabularDataset(test).drop('Id', axis=1)
test_data.head()


In [ ]:
metric = 'accuracy'
time=1*3600
predictor = TabularPredictor(label='Cover_Type', eval_metric=metric).fit(
    train_data= train_data, time_limit=time, 
    ag_args_fit={'num_gpus': 1},
    presets='best_quality', verbosity=2)
predictor.leaderboard(train_data, silent=True)


In [ ]:
results = predictor.fit_summary()

In [ ]:
y_pred = predictor.predict(test_data)
y_pred.head(5)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
submission.Cover_Type = y_pred
submission.to_csv('submission.csv', index=False)